In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier
from sklearn.impute import KNNImputer

In [2]:
application_data = pd.read_csv("application_train.csv")
application_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
application_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [4]:
application_data.shape

(307511, 122)

In [5]:
application_data.columns.tolist()

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_A

In [6]:
missing_values = application_data.isnull().sum().sort_values(ascending=False)
missing_values

COMMONAREA_MEDI             214865
COMMONAREA_AVG              214865
COMMONAREA_MODE             214865
NONLIVINGAPARTMENTS_MODE    213514
NONLIVINGAPARTMENTS_AVG     213514
                             ...  
NAME_HOUSING_TYPE                0
NAME_FAMILY_STATUS               0
NAME_EDUCATION_TYPE              0
NAME_INCOME_TYPE                 0
SK_ID_CURR                       0
Length: 122, dtype: int64

In [7]:
missing_percentage = (missing_values / len(application_data)) * 100
missing_percentage

COMMONAREA_MEDI             69.872297
COMMONAREA_AVG              69.872297
COMMONAREA_MODE             69.872297
NONLIVINGAPARTMENTS_MODE    69.432963
NONLIVINGAPARTMENTS_AVG     69.432963
                              ...    
NAME_HOUSING_TYPE            0.000000
NAME_FAMILY_STATUS           0.000000
NAME_EDUCATION_TYPE          0.000000
NAME_INCOME_TYPE             0.000000
SK_ID_CURR                   0.000000
Length: 122, dtype: float64

In [8]:
missing_data =pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
print(missing_data[missing_data['Missing Values'] > 0])

                          Missing Values  Percentage
COMMONAREA_MEDI                   214865   69.872297
COMMONAREA_AVG                    214865   69.872297
COMMONAREA_MODE                   214865   69.872297
NONLIVINGAPARTMENTS_MODE          213514   69.432963
NONLIVINGAPARTMENTS_AVG           213514   69.432963
...                                  ...         ...
EXT_SOURCE_2                         660    0.214626
AMT_GOODS_PRICE                      278    0.090403
AMT_ANNUITY                           12    0.003902
CNT_FAM_MEMBERS                        2    0.000650
DAYS_LAST_PHONE_CHANGE                 1    0.000325

[67 rows x 2 columns]


In [9]:
columns_to_keep=missing_percentage[missing_percentage<=58].index
application_data=application_data[columns_to_keep]
application_data.shape

(307511, 99)

In [10]:
columns_to_impute_20 = missing_percentage[missing_percentage <= 20].index
for column in columns_to_impute_20:
    if application_data[column].dtype in ['int64', 'float64']:  
        median_value = application_data[column].median()
        application_data.loc[:, column] = application_data[column].fillna(median_value) 
    else:  
        mode_value = application_data[column].mode()[0] 
        application_data.loc[:, column] = application_data[column].fillna(mode_value)  
print(application_data[columns_to_impute_20].isnull().sum())

EXT_SOURCE_3                  0
AMT_REQ_CREDIT_BUREAU_HOUR    0
AMT_REQ_CREDIT_BUREAU_DAY     0
AMT_REQ_CREDIT_BUREAU_WEEK    0
AMT_REQ_CREDIT_BUREAU_MON     0
                             ..
NAME_HOUSING_TYPE             0
NAME_FAMILY_STATUS            0
NAME_EDUCATION_TYPE           0
NAME_INCOME_TYPE              0
SK_ID_CURR                    0
Length: 72, dtype: int64


In [31]:
from sklearn.impute import IterativeImputer, SimpleImputer


columns_to_impute_20_58 = missing_percentage[(missing_percentage > 20) & (missing_percentage <= 58)].index

# تحديد الأعمدة الرقميةnumerical_cols_20_58 = application_data[columns_to_impute_20_58].select_dtypes(include=['int64', 'float64']).columns

# تحديد الأعمدة الفئوية
categorical_cols_20_58 = application_data[columns_to_impute_20_58].select_dtypes(include=['object', 'category']).columns

# "Imputing numerical values using Iterative Imputer"
# زيادة عدد التكرارات إلى 50 أو أكثر

iterative_imputer = IterativeImputer(max_iter=100, random_state=0, n_nearest_features=10, imputation_order='ascending')
application_data.loc[:, numerical_cols_20_58] = iterative_imputer.fit_transform(application_data[numerical_cols_20_58])

# "Imputing categorical values using simple Imputer"
mode_imputer = SimpleImputer(strategy="most_frequent")
application_data.loc[:, categorical_cols_20_58] = mode_imputer.fit_transform(application_data[categorical_cols_20_58])

# التحقق من القيم المفقودة بعد المعالجة
print(application_data[columns_to_impute_20_58].isnull().sum())

KeyError: "['HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE'] not in index"

In [12]:
print("Total missing values after imputing:", application_data.isnull().sum().sum())
application_data.shape

Total missing values after imputing: 0


(307511, 99)

In [13]:
# Step 1: Drop the TARGET column
application_data = application_data.drop(columns=['TARGET'])

# Step 2: Function to check and drop columns with repeated values exceeding the threshold
def check_and_drop_repeated_columns(data, threshold=0.8):
    repeated_columns = []  # List to store columns to be dropped
    for column in data.columns:
        # Calculate the percentage of the most frequent value
        most_common_value_count = data[column].value_counts(normalize=True).max()
        if most_common_value_count > threshold:
            repeated_columns.append(column)  # Add column to drop list
    # Drop the repeated columns
    data = data.drop(columns=repeated_columns)
    return data

# Step 3: Apply the function on application_data
application_data = check_and_drop_repeated_columns(application_data)

# Step 4: Final shape of application_data
print(f"Shape of application_data: {application_data.shape}")

Shape of application_data: (307511, 57)


In [14]:
# Reload the original data to extract the TARGET column
original_data = pd.read_csv('application_train.csv')

# Add the TARGET column back to train_data
application_data['TARGET'] = original_data['TARGET']

# Verify the updated shape
print(f"Shape of train_data after adding TARGET: {application_data.shape}")

Shape of train_data after adding TARGET: (307511, 58)


In [15]:
print(application_data.columns.tolist() )    
print('-------------------------------------------')
print('shape = ',application_data.shape[1])

['EXT_SOURCE_1', 'NONLIVINGAREA_MODE', 'NONLIVINGAREA_AVG', 'NONLIVINGAREA_MEDI', 'ELEVATORS_MEDI', 'ELEVATORS_AVG', 'ELEVATORS_MODE', 'WALLSMATERIAL_MODE', 'APARTMENTS_MEDI', 'APARTMENTS_AVG', 'APARTMENTS_MODE', 'ENTRANCES_MEDI', 'ENTRANCES_AVG', 'ENTRANCES_MODE', 'LIVINGAREA_AVG', 'LIVINGAREA_MODE', 'LIVINGAREA_MEDI', 'FLOORSMAX_MODE', 'FLOORSMAX_MEDI', 'FLOORSMAX_AVG', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BEGINEXPLUATATION_AVG', 'TOTALAREA_MODE', 'OCCUPATION_TYPE', 'EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'EXT_SOURCE_2', 'AMT_GOODS_PRICE', 'AMT_ANNUITY', 'CNT_FAM_MEMBERS', 'DAYS_LAST_PHONE_CHANGE', 'CNT_CHILDREN', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_DOCUMENT_3', 'FLAG_OWN_REALTY', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'FLAG_PHONE', 'REG_CITY_NOT_WORK_CITY', 'HOUR_APPR_PROCESS_START', 'WEEKDAY_APPR_PROCESS_START', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'ORGANIZATION_TYPE', 'DAY

# outliers

In [17]:
numeric_columns = application_data.select_dtypes(include=['float64', 'int64']).columns
for column in numeric_columns:
    Q1 = application_data[column].quantile(0.25)
    Q3 = application_data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers_condition = (application_data[column] < lower_bound) | (application_data[column] > upper_bound)
    num_outliers = outliers_condition.sum()
    
    if num_outliers > 0:
        print(f"Column '{column}' has {num_outliers} outliers.")

Column 'EXT_SOURCE_1' has 102844 outliers.
Column 'NONLIVINGAREA_MODE' has 18954 outliers.
Column 'NONLIVINGAREA_AVG' has 19872 outliers.
Column 'NONLIVINGAREA_MEDI' has 19806 outliers.
Column 'ELEVATORS_MEDI' has 24514 outliers.
Column 'ELEVATORS_AVG' has 24538 outliers.
Column 'ELEVATORS_MODE' has 22770 outliers.
Column 'APARTMENTS_MEDI' has 59834 outliers.
Column 'APARTMENTS_AVG' has 57540 outliers.
Column 'APARTMENTS_MODE' has 66087 outliers.
Column 'ENTRANCES_MEDI' has 119296 outliers.
Column 'ENTRANCES_AVG' has 118719 outliers.
Column 'ENTRANCES_MODE' has 121870 outliers.
Column 'LIVINGAREA_AVG' has 50675 outliers.
Column 'LIVINGAREA_MODE' has 52709 outliers.
Column 'LIVINGAREA_MEDI' has 50570 outliers.
Column 'FLOORSMAX_MODE' has 79762 outliers.
Column 'FLOORSMAX_MEDI' has 72416 outliers.
Column 'FLOORSMAX_AVG' has 71737 outliers.
Column 'YEARS_BEGINEXPLUATATION_MODE' has 36360 outliers.
Column 'YEARS_BEGINEXPLUATATION_MEDI' has 35976 outliers.
Column 'YEARS_BEGINEXPLUATATION_AV